# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time

#Spicy stats is used for regressions
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
#I have no idea what this is or if I should leave it
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

620

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).

> **HINT:** The OpenWeatherMap API only allows 60 calls per minute under their free plan. Try using `time.sleep(60)` after each set of 50 cities to avoid API errors due to large calls.

In [3]:
#Print to see all the cities within my list
cities[0:651]

['ushuaia',
 'batemans bay',
 'rikitea',
 'belushya guba',
 'taolanaro',
 'necochea',
 'hatsukaichi',
 'pontianak',
 'san patricio',
 'sangar',
 'shawville',
 'kruisfontein',
 'ferme-neuve',
 'varhaug',
 'mataura',
 'barentsburg',
 'tezu',
 'avera',
 'barrow',
 'kembe',
 'bambous virieux',
 'severo-kurilsk',
 'chuy',
 'labuhan',
 'kamenskoye',
 'atuona',
 'nova olimpia',
 'champerico',
 'vaini',
 'butaritari',
 'luanda',
 'narsaq',
 'nizhneyansk',
 'pochutla',
 'jiehu',
 'nikolskoye',
 'hithadhoo',
 'mount gambier',
 'busselton',
 'carnarvon',
 'mahebourg',
 'kathu',
 'port blair',
 'virginia beach',
 'alofi',
 'saint charles',
 'fairbanks',
 'jiangyou',
 'sentyabrskiy',
 'barabai',
 'la ronge',
 'saryshagan',
 'jamestown',
 'klaksvik',
 'yellandu',
 'manmad',
 'richards bay',
 'praya',
 'vaitupu',
 'tsihombe',
 'tasiilaq',
 'ribeira grande',
 'matamoros',
 'ilulissat',
 'bodinayakkanur',
 'presidencia roque saenz pena',
 'los alamos',
 'korla',
 'sembe',
 'vardo',
 'arraial do cabo',


In [4]:
# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

In [5]:
 # Build query URL and request your results in Celsius
#Call any city name in my list
city = "takaka"
query_url = f"{url}appid={weather_api_key}&q={city}&units={units}"

# Get weather data
weather_response = requests.get(query_url)
print(weather_response.status_code)
weather_json = weather_response.json()
#Print 200 means it's good

200


In [6]:
#Find my types to make my keys from the dictionary

weather_json.keys()

dict_keys(['coord', 'weather', 'base', 'main', 'visibility', 'wind', 'clouds', 'dt', 'sys', 'timezone', 'id', 'name', 'cod'])

In [7]:
#Print the weather json to find each location(s) lon, lat to use informantion in constructing DataFrame
#This will pull the keys inforamtion of the city I called above 'takaka'

print(weather_json)

{'coord': {'lon': 172.8, 'lat': -40.85}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 57.9, 'feels_like': 56.77, 'temp_min': 57.9, 'temp_max': 57.9, 'pressure': 1028, 'humidity': 72, 'sea_level': 1028, 'grnd_level': 1026}, 'visibility': 10000, 'wind': {'speed': 3.83, 'deg': 72, 'gust': 3.2}, 'clouds': {'all': 60}, 'dt': 1667680326, 'sys': {'country': 'NZ', 'sunrise': 1667668258, 'sunset': 1667718830}, 'timezone': 46800, 'id': 2181997, 'name': 'Takaka', 'cod': 200}


In [8]:
# Begin to parse the data

lat = weather_json["coord"]["lat"]
lon = weather_json["coord"]["lon"]
country = weather_json["sys"]["country"]
cloudiness = weather_json["clouds"]["all"]

date = weather_json["dt"]
city_name = weather_json["name"]
wind = weather_json["wind"]["speed"]

temp = weather_json["main"]["temp"]
pressure = weather_json["main"]["pressure"]
humidity = weather_json["main"]["humidity"]



In [9]:
#Check to see if the above information is calling and getting the correct numbers
pressure

1028

In [10]:
lats = []
lngs = []
countries = []
clouds = []
dates = []
names = []
winds = []
temps = []
pressures = []
humids = []

for city in cities[0:60]:
    
    # make request
    print(city)
    
    # Build query URL and request your results in Celsius
    query_url = f"{url}appid={weather_api_key}&q={city}&units={units}"

    # Get weather data
    weather_response = requests.get(query_url)
    print(weather_response.status_code)
    print()
    #Use except/try to catch errors
    try:
        # get data
        weather_json = weather_response.json()

        # parse the data
        #Callables equal the cell above, pretty easy to remember
        lat = weather_json["coord"]["lat"]
        lng = weather_json["coord"]["lon"]
        country = weather_json["sys"]["country"]
        cloudiness = weather_json["clouds"]["all"]

        date_dt = weather_json["dt"]
        city_name = weather_json["name"]
        wind = weather_json["wind"]["speed"]

        temp = weather_json["main"]['temp']
        pressure = weather_json["main"]['pressure']
        humidity = weather_json["main"]['humidity']

        # Create a lists
        lats.append(lat)
        lngs.append(lng)
        countries.append(country)
        clouds.append(cloudiness)
        dates.append(date_dt)
        names.append(city_name)
        winds.append(wind)
        temps.append(temp)
        pressures.append(pressure)
        humids.append(humidity)
     #Use except/try to catch errors       
    except Exception as e:
        print(e)
        
        # Create a lists
        lats.append(np.nan)
        lngs.append(np.nan)
        countries.append(None)
        clouds.append(np.nan)
        dates.append(np.nan)
        names.append(city)
        winds.append(np.nan)
        temps.append(np.nan)
        pressures.append(np.nan)
        humids.append(np.nan)
    #How can we ensure we are setting our sleep function to a sufficiant enough time?    
    time.sleep(1)

ushuaia
200

batemans bay
200

rikitea
200

belushya guba
404

'coord'
taolanaro
404

'coord'
necochea
200

hatsukaichi
200

pontianak
200

san patricio
200

sangar
200

shawville
200

kruisfontein
200

ferme-neuve
200

varhaug
200

mataura
200

barentsburg
404

'coord'
tezu
200

avera
200

barrow
200

kembe
200

bambous virieux
200

severo-kurilsk
200

chuy
200

labuhan
200

kamenskoye
404

'coord'
atuona
200

nova olimpia
200

champerico
200

vaini
200

butaritari
200

luanda
200

narsaq
200

nizhneyansk
404

'coord'
pochutla
200

jiehu
200

nikolskoye
200

hithadhoo
200

mount gambier
200

busselton
200

carnarvon
200

mahebourg
200

kathu
200

port blair
200

virginia beach
200

alofi
200

saint charles
200

fairbanks
200

jiangyou
200

sentyabrskiy
404

'coord'
barabai
200

la ronge
200

saryshagan
404

'coord'
jamestown
200

klaksvik
200

yellandu
200

manmad
200

richards bay
200

praya
200

vaitupu
404

'coord'
tsihombe
404

'coord'


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [11]:
#Create a DataFrame

wdf = pd.DataFrame()

wdf["City"] = names
wdf["Country"] = countries
wdf["Latitude"] = lats
wdf["Longitude"] = lngs
wdf["Date"] = dates
wdf["Wind_Speed"] = winds
wdf["Pressure"] = pressures
wdf["Humidity"] = humids
wdf["Temperature"] = temps
wdf["CLoudiness"] = clouds

wdf.head(200)



,City,Country,Latitude,Longitude,Date,Wind_Speed,Pressure,Humidity,Temperature,CLoudiness
0,Ushuaia,AR,-54.8000,-68.3000,1.667680e+09,5.75,1008.0,39.0,64.06,0.0
1,Batemans Bay,AU,-35.7167,150.1833,1.667680e+09,3.02,1025.0,86.0,60.10,4.0
2,Rikitea,PF,-23.1203,-134.9692,1.667680e+09,17.27,1017.0,73.0,70.77,87.0
3,belushya guba,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,taolanaro,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Necochea,AR,-38.5473,-58.7368,1.667680e+09,12.75,1021.0,46.0,72.84,66.0
6,Hatsukaichi,JP,34.3500,132.3333,1.667680e+09,5.03,1023.0,83.0,51.75,16.0
7,Pontianak,ID,-0.0333,109.3333,1.667680e+09,6.91,1011.0,94.0,76.98,75.0
8,San Patricio,US,28.0170,-97.5169,1.667680e+09,8.97,1012.0,37.0,75.42,3.0
9,Sangar,RU,63.9242,127.4739,1.667680e+09,1.66,1006.0,100.0,-6.05,98.0


In [12]:
#Make DataFrame to a CSV file
wdf.to_csv("bh_data.csv", index=False)

In [21]:
wdf.describe()

,Latitude,Longitude,Date,Wind_Speed,Pressure,Humidity,Temperature,CLoudiness
count,55.000000,55.000000,5.500000e+01,55.000000,55.000000,55.000000,55.000000,55.000000
mean,0.310796,7.212744,1.667535e+09,8.616182,1015.054545,72.927273,59.529818,51.109091
std,36.640723,101.055656,6.309088e+01,6.608096,6.878356,18.329146,20.012855,37.449558
min,-54.800000,-175.200000,1.667535e+09,0.400000,1000.000000,21.000000,-0.060000,0.000000
25%,-29.829900,-73.589100,1.667535e+09,3.615000,1011.000000,66.000000,50.285000,12.500000
50%,-9.085300,15.159400,1.667535e+09,7.090000,1014.000000,75.000000,62.110000,53.000000
75%,34.303550,110.745700,1.667535e+09,11.500000,1019.000000,84.500000,73.135000,86.000000
max,77.484000,172.800000,1.667535e+09,34.360000,1032.000000,99.000000,90.970000,100.000000


In [22]:
#Should I drop the null values in city? or should I leave them..its only five values..
wdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City         60 non-null     object 
 1   Country      55 non-null     object 
 2   Latitude     55 non-null     float64
 3   Longitude    55 non-null     float64
 4   Date         55 non-null     float64
 5   Wind_Speed   55 non-null     float64
 6   Pressure     55 non-null     float64
 7   Humidity     55 non-null     float64
 8   Temperature  55 non-null     float64
 9   CLoudiness   55 non-null     float64
dtypes: float64(8), object(2)
memory usage: 4.8+ KB


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression